## What is this notebook?

> This notebook is from [here](https://colab.research.google.com/drive/1OeH5Jq_WRmSNLuq0ApS5iSsUXLvCZFbW?usp=sharing)


This notebook showcase how easy and user friendly wandb is. It is from [Weights & Biases](https://wandb.ai/site).


------


## Installation

First, visit [pytorch](https://pytorch.org/get-started/locally/) and install torch dependency according to your environment.

Then install below libraries.




In [4]:
!pip3 install torch torchvision -q
!pip install wandb onnx numpy tqdm  -q


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


### Start local server

First start a local server. You will also need to follow [this guide](https://docs.wandb.ai/guides/hosting/how-to-guides/basic-setup) to get the free license.


After getting the free license, you can view the license again [here](https://deploy.wandb.ai/). (Because the wandb platform is not clear how to find it.)

Start your docker daemon and run below command

In [ ]:
!wandb server start

### Code execution

After the server started, we can start coding and log the metadata to local wandb.

In [1]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torchvision
from pathlib import Path
from torch.utils.data import TensorDataset,DataLoader

from tqdm.auto import tqdm

# Setting deterministic behavior, i.e. no randomness
torch.backends.cudnn.deterministic = True
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
torch.manual_seed(hash("by removing stochasticity") % 2**32 - 1)
torch.cuda.manual_seed_all(hash("so runs are repeatable") % 2**32 - 1)

# Setting up device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device: ",device)

# Removing slow mirrors of MNIST mirror
torchvision.datasets.MNIST.mirrors = [mirror for mirror in torchvision.datasets.MNIST.mirrors
                                      if not mirror.startswith("http://yann.lecun.com")]


device:  cpu


/Users/leung.tsz.kit/Desktop/work/code/test/github_repos/wandb_trial/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Import and Log in to W&B
First, create an account / login on the W&B [homepage](https://www.wandb.jp/) 

You'll be asked for an API key during the initial login. You can find your API keys by clicking the person icon in the top right corner of the W&B screen, pressing User setting, and scrolling down.

You can set it in environment variable or run `login`` directly

In [2]:
# Specifying the project name on wandb platform for tracking
import datetime
now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
project_name = now + "local-example-mnist-pytorch"

# This is secret and shouldn't be checked into version control
# os.environ["WANDB_API_KEY"] = "<your key here>"
os.environ["WANDB_API_KEY"] = "local-9c0735887ffa34362b304788f06fd6a6087c5efd"
os.environ["WANDB_BASE_URL"] = "http://localhost:8080"

# This is where wandb store local files
LOCAL_TMP_WANDB_DIR = Path("./tmp/wandb").joinpath(project_name)
os.makedirs(LOCAL_TMP_WANDB_DIR, exist_ok=True)
os.environ["WANDB_DIR"] = str(LOCAL_TMP_WANDB_DIR)

print("Wandb will store everything in : ", LOCAL_TMP_WANDB_DIR)

Wandb will store everything in :  tmp/wandb/20230814_145417local-example-mnist-pytorch


In [3]:
# logging in
import wandb
import os
wandb.login()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: keith. Use `wandb login --relogin` to force relogin


True

-----

## Managing Data with wandb Integration and Artifacts

First, we will prepare the data in the following steps.


1.  `load_and_log`: Import the mnist data, divide the data into train/validation/test.

    - The process itself is standard Pytorch workflow, but the only difference from the standard pipeline is that everything is executed within the wandb.init context (there is another way to upload data to wandb by using wandb.finish at the end of all codes instead of using with clause, but by using with clause, the wandb process ends without forgetting even when an error occurs, and necessary logs are linked to wandb, so we recommend using with clause as a best practice).
    - Artifacts are useful for versioning data. By linking to wandb which data was used in the process of training and inference of the model using Artifacts, the experimental process is organized, and reproducibility also improves.
2.  `preprocess_and_log`:: Perform preprocessing such as normalization.
    - Original data ('mnist-raw:latest') is uploaded to Artifact, and before processing the data, specify which data on Artifact to process using run.use_artifact('mnist-raw:latest'). Going through this process and saving the processed data back to Artifact allows you to check the lineage (relationship) of the processed data, i.e., which data it was derived from, on W&B.





Before defining the above functions, here are some basic ML code to load/process data.

In [4]:
# This preprocess function is used to preprocess the data before training the model
def preprocess(dataset, normalize=True, expand_dims=True):
  x, y = dataset.tensors
  if normalize:
      # Scale the image to a range of [0, 1]
      x = x.type(torch.float32) / 255
  if expand_dims:
      # Change the shape of the image to (1, 28, 28)
      x = torch.unsqueeze(x, 1)
  return TensorDataset(x, y)


# This read function is used to read the dataset
def read(data_dir, split):
  filename = split + ".pt"
  x, y = torch.load(os.path.join(data_dir, filename))
  return TensorDataset(x, y)


# This load function is used to load and split the MNIST dataset into train, val, test
def load(train_size=50000,slice=50):
  # Split the data into train and test
  train = torchvision.datasets.MNIST("./tmp/", train=True, download=True)
  test = torchvision.datasets.MNIST("./tmp/", train=False, download=True)
  (x_train, y_train), (x_test, y_test) = (train.data, train.targets), (test.data, test.targets)

  # Split train into train and validation
  x_train, x_val = x_train[:train_size], x_train[train_size:]
  y_train, y_val = y_train[:train_size], y_train[train_size:]

  # Extract a portion of the data for hands-on use
  training_set = torch.utils.data.Subset(TensorDataset(x_train, y_train), indices=range(0, len(y_train), slice))
  validation_set = torch.utils.data.Subset(TensorDataset(x_val, y_val), indices=range(0, len(y_val), slice))
  test_set = torch.utils.data.Subset(TensorDataset(x_test, y_test), indices=range(0, len(y_test), slice))

  datasets = [training_set, validation_set, test_set]
  return datasets


# This transform function is used to transform the dataset into TensorDataset
def transform_subset2tensordataset(subset):
  # Convert subdataset to TensorDataset for saving to Artifact
  subset = [subset[i] for i in range(len(subset))]
  images = torch.stack([item[0] for item in subset])
  labels = torch.tensor([item[1] for item in subset])
  data = TensorDataset(images, labels)
  return data

In [5]:
def load_and_log():
    # Create a "run" in wandb
    # job_type is a free text field you can use to group your runs
    with wandb.init(project=project_name, job_type="load-data") as run:
        # Definte the Run name
        run.name = "load-data-" + run.name

        datasets = load()  # separate code for loading the datasets
        names = ["training", "validation", "test"]

        # Define an Artifact
        raw_data = wandb.Artifact(
            "mnist-raw", type="dataset",
            description="Raw MNIST dataset, split into train/val/test",
            metadata={"source": "torchvision.datasets.MNIST","sizes": [len(dataset) for dataset in datasets]}
            )

        # Save each subset to pt file and add to Artifact
        for name, subset in zip(names, datasets):
            # Convert subdataset to TensorDataset for saving to Artifact
            data = transform_subset2tensordataset(subset)
            with raw_data.new_file(name + ".pt", mode="wb") as file:
                x, y = data.tensors
                torch.save((x, y), file)

        # Save Artifact to W&B
        run.log_artifact(raw_data)


def preprocess_and_log(steps):
    #　Initialize wandb
    with wandb.init(project=project_name, job_type="preprocess-data") as run:
        # Definte the Run name
        run.name = "preprocess-data-" + run.name

        # Define an Artifact
        processed_data = wandb.Artifact(
            "mnist-preprocess", type="dataset",
            description="Preprocessed MNIST dataset",
            metadata=steps)

        # Define which Artifact to use and download if necessary
        # This is how lineage is established between runs
        raw_data_artifact = run.use_artifact('mnist-raw:v0')
        raw_dataset = raw_data_artifact.download()

        for split in ["training", "validation", "test"]:
            raw_split = read(raw_dataset, split)
            processed_dataset = preprocess(raw_split, **steps)
            # After preprocessing, save to Artifact
            with processed_data.new_file(split + ".pt", mode="wb") as file:
                x, y = processed_dataset.tensors
                torch.save((x, y), file)

        run.log_artifact(processed_data)

Now, we have the function ready for the two purpose:
- Load MNIST dataset from source (pytorch), save to W&B Artifact
- Download artifact from W&B, preprocess it, and save to W&B Artifact.

Next, let's execute it.

In [6]:
load_and_log()
steps = {"normalize": True, "expand_dims": True}
preprocess_and_log(steps)

wandb:   3 of 3 files downloaded.  


----------


## EDA with Table

W&B's Table enables Exploratory Data Analysis (EDA) using rich visualization capabilities. In this case, the data used is stored in W&B's table format for EDA on the W&B platform.

In [7]:
# Create a run in wandb
with wandb.init(project=project_name, job_type="EDA") as run:
  # Define Run name
  run.name = "EDA-" + run.name

  #  Create a wandb Table (instead of artifact)
  table = wandb.Table(columns=["image", "label","split"])

  # Define which Artifact to use and download if necessary
  # Again, it is how artifacts are linked in lineage
  raw_data_artifact = run.use_artifact('mnist-raw:latest')
  raw_dataset = raw_data_artifact.download()

  for split in ["training", "validation", "test"]:
    # Note: we are using raw MNIST data
    raw_split = read(raw_dataset, split) #<= TensorDataset
    data_loader = DataLoader(raw_split, batch_size=100, shuffle=False)
    for _, (imgs, labels) in enumerate(tqdm(data_loader,leave=False)):
      for img, label in zip(imgs.to("cpu"), labels.to("cpu")):
        w_img = wandb.Image(img.numpy())
        # Store image, label, and split name in the wandb table
        table.add_data(w_img, label, split)

  # Upload the table
  run.log({'EDA_table_before_training':table})

wandb:   3 of 3 files downloaded.  


-----

## Model Training

Next, let's proceed with model training. Firstly, we define the overall pipeline, `model_pipeline`. The only difference from a standard pipeline is that everything is performed within the context of wandb.init. Just like when we were "preparing the data", calling this function establishes a communication path between the code and our servers.

The difference from before is that we're defining a config. In a typical workflow, you store hyperparameters and metadata in a `config` dictionary and access them as needed. Inside `model_pipeline`, we're doing the following:

1. `make`: Here, we define the model architecture, set up data loaders, loss functions, and optimization methods. This is essentially the same as the regular Pytorch process.
    - By using the Artifact we saved earlier, we can record in the Lineage which data was used for the training of this model.
2. `train`: Here, we train the model and validate it against the validation data.
    - Here, two functions from wandb, `watch` and `log`, come into play.
      - `wandb.watch` is a function that can only be used when integrating with Pytorch. It logs the gradients and parameters of the model every log_freq steps of training. Before starting the training, you simply call `wandb.watch`. The rest of the training code remains the same.
      - `wandb.log` allows you to log any information, such as metrics. You can also optionally log which step of the training it is.




Let's create a pipeline, manage metadata and hyperparameters with `wandb.init`


In [8]:
# Conventional and convolutional neural network
class ConvNet(nn.Module):
    def __init__(self, kernels, dropout, classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, kernels[0], kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(kernels[0], kernels[1], kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7 * 7 * kernels[-1], classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

# This function relies on the `run` arg, so it will be called within a "with" statement.
def make(config, run):
    # Define which Artifact to use and download it if necessary
    # Again, it decides the linkage between artifacts in lineage
    raw_data_artifact = run.use_artifact('mnist-preprocess:v0', type='dataset')
    raw_dataset = raw_data_artifact.download()

    # Create data loaders for preprocessed MNIST dataset
    train_loader = DataLoader(read(raw_dataset, "training"), config.batch_size,
                                shuffle=False, pin_memory=True, num_workers=2)
    validation_loader = DataLoader(read(raw_dataset, "validation"), config.batch_size,
                                shuffle=False, pin_memory=True, num_workers=2)
    test_loader = DataLoader(read(raw_dataset, "test"), config.batch_size,
                                shuffle=False, pin_memory=True, num_workers=2)

    # Build the model architecture
    model = ConvNet(config.kernels,  config.dropout, config.classes).to(device)
    # Define the loss function
    criterion = nn.CrossEntropyLoss()

    # Set the optimization method
    if config.optimizer == "sgd":
        optimizer = torch.optim.SGD(model.parameters(),lr=config.learning_rate, momentum=0.9)
    elif config.optimizer == "adam":
        optimizer = torch.optim.Adam(model.parameters(),lr=config.learning_rate)

    return model, train_loader, validation_loader, test_loader, criterion, optimizer


In [12]:
def train(model, training_loader, validation_loader, criterion, optimizer, config, run):
    #　Track model gradients, weights, etc. using wandb.watch
    # wandb.watch : Hook into the torch model to collect gradients and the topology.
    # log_freq :  every N batches
    wandb.watch(model, criterion, log="all", log_freq=5)

    # Start the learning
    table = wandb.Table(columns=["image", "pred", "target"]+[f"score_{i}" for i in range(10)])
    example_ct = 0
    for epoch in tqdm(range(config.epochs)):
        for step, (images, labels) in enumerate(training_loader):
            loss = train_batch(images, labels, model, optimizer, criterion)
            example_ct +=  len(images)
            #　Track the model's loss using run.log
            # Log a dictionary of data to the current run's history.
            run.log({"epoch": epoch, "train_loss": loss}, step=example_ct, commit=False)
        # Do validation at the end of each epoch
        validation(model, validation_loader, criterion,  run, table, log_images=config.log_prediction_iamge)


# Train the model for one batch
def train_batch(images, labels, model, optimizer, criterion):
    images, labels = images.to(device), labels.to(device)

    # Forward pass ➡
    outputs = model(images)
    loss = criterion(outputs, labels)
    # Backward pass ⬅
    optimizer.zero_grad()
    loss.backward()
    # Optimization step
    optimizer.step()
    # Return for wandb logging
    return loss


# Calculate the validation loss and accuracy, and log to wandb
def validation(model, validation_loader, criterion, run, table,  log_images):
    model.eval()
    with torch.no_grad():
        correct, total, val_loss = 0, 0, 0
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            val_loss += criterion(outputs, labels)*labels.size(0)
            if log_images:
                log_image_table(images, predicted, labels, outputs.softmax(dim=1), run, table)
        run.log({"validation_loss":val_loss / total ,"validation_accuracy": correct / total})


def log_image_table(images, predicted, labels, probs, run, table):
    # Create a wandb Table and save the images, predicted values, and measured values
    for img, pred, targ, prob in zip(images.to("cpu"), predicted.to("cpu"), labels.to("cpu"), probs.to("cpu")):
        table.add_data(wandb.Image(img[0].numpy()*255), pred, targ, *prob.numpy())
    run.log({"validation_prediction_table":table}, commit=False)

Then use all above functions to build the end2end pipeline

In [13]:
def model_pipeline(config):
    #　Create a wandb run
    with wandb.init(project=project_name, config=config, job_type="training") as run:
        # Define the name of the run
        run.name = "training-" + run.name
        config = wandb.config
        # Define model architecture, set up data loaders, loss function, and optimization methods
        model, train_loader, validation_loader,_, criterion, optimizer = make(config, run)
        # Train the model and validate it with the validation data
        train(model, train_loader,validation_loader, criterion, optimizer, config, run)

        #　Save the model to the Artifact after training loop
        artifact = wandb.Artifact('model', type='model')
        with  artifact.new_file('model.pt',mode='wb') as file:
            torch.save(model.state_dict(), file)
        # Upload to wandb
        run.log_artifact(artifact)

### Model Pipeline Execution

Now that we have defined the entire pipeline and added some W&B code, we're ready to run our tracked experiments. When you run your code using W&B's tracking feature, the following links will be outputted:
*   Document
*   Project page (a page that organizes all runs within the project)
*   Page where the results of the execution are saved


Go to the Run page to see the tracked results.


In [14]:
# Some hyperparameters for the model, just for demo purposes
config = dict(
    epochs=10,
    kernels=[16, 32],
    batch_size=128,
    dropout =0.1,
    learning_rate=0.005,
    optimizer="adam",
    log_prediction_iamge=True,
    classes=10,
    dataset="MNIST",
    architecture="CNN"
)

In [15]:
model_pipeline(config)

wandb:   3 of 3 files downloaded.  
100%|██████████| 10/10 [01:06<00:00,  6.70s/it]


epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇████
train_loss,██▇▅▄▃▃▂▂▂▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_accuracy,▁▄▅▇▇▇▇██▇
validation_loss,█▄▃▂▁▂▁▁▂▂
epoch,9
train_loss,0.02267
validation_accuracy,0.935
validation_loss,0.28044


------

## Hyperparameter Tuning

A Sweep is a tool for optimizing hyperparameters and models. With a few lines of code, you can conduct hyperparameter tuning. First, define the requirements of the hyperparameters to be optimized. Then, choose the method of hyperparameter optimization (random search or grid search) and select the metric to optimize.





We will do hyperparameter Tuning Using Sweep

In [16]:
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'validation_loss',
        'goal': 'minimize'
        },
    }
parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'dropout': {
          'values': [0.1, 0.2, 0.3]
        },
    'learning_rate': {
        # Generate random numbers uniformly distributed between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
      },
    'batch_size': {
        # Generate random numbers from a distribution equally spaced on a logarithmic scale between 32 and 256
        'distribution': 'q_log_uniform_values',
        'q': 8,
        'min': 32,
        'max': 256,
      },
    'kernels':{
        'values':[[16,32],[32,64]]
      },
    'epochs':{
        'value':5
    },
    'classes':{
        'value':10,
    },
    'dataset':{
        'value':'MNIST',
    },
    'architecture':{
        'value':'CNN',
    },
    'log_prediction_iamge':{
        'value':False
    },
    }

sweep_config['parameters'] = parameters_dict

#🐝 Create a function for Sweep
def model_pipeline_sweep(config=None):
    #🐝　Create a run with wandb
    with wandb.init(config=config) as run:
      config = run.config
      # Define the model architecture, set up the dataloader, loss function, and optimization method
      model, train_loader, validation_loader, _,criterion, optimizer = make(config, run)
      # Train the model and validate it against validation data
      train(model, train_loader, validation_loader, criterion, optimizer, config, run)
      #🐝　Save the model to Artifact
      artifact = wandb.Artifact('model', type='model')
      with  artifact.new_file('model.pt',mode='wb') as file:
          torch.save(model.state_dict(), file)
      run.log_artifact(artifact)

In [17]:
sweep_id = wandb.sweep(sweep_config, project=project_name)
wandb.agent(sweep_id=sweep_id, function=model_pipeline_sweep, count=5)

Create sweep with ID: 6u6nmhv5
Sweep URL: http://localhost:8080/keith/20230814_145417local-example-mnist-pytorch/sweeps/6u6nmhv5


wandb: Agent Starting Run: 62iraj7o with config:
wandb: 	architecture: CNN
wandb: 	batch_size: 32
wandb: 	classes: 10
wandb: 	dataset: MNIST
wandb: 	dropout: 0.2
wandb: 	epochs: 5
wandb: 	kernels: [32, 64]
wandb: 	learning_rate: 0.05920082884401609
wandb: 	log_prediction_iamge: False
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb:   3 of 3 files downloaded.  
100%|██████████| 5/5 [00:34<00:00,  6.94s/it]


epoch,▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆████████
train_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_accuracy,▁▁▁▁▁
validation_loss,█▁▁▁▁
epoch,4
train_loss,2.36617
validation_accuracy,0.11
validation_loss,2.30826


wandb: Agent Starting Run: aeqa7vkp with config:
wandb: 	architecture: CNN
wandb: 	batch_size: 160
wandb: 	classes: 10
wandb: 	dataset: MNIST
wandb: 	dropout: 0.1
wandb: 	epochs: 5
wandb: 	kernels: [16, 32]
wandb: 	learning_rate: 0.004889491678429114
wandb: 	log_prediction_iamge: False
wandb: 	optimizer: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


...

wandb:   3 of 3 files downloaded.  
100%|██████████| 5/5 [00:35<00:00,  7.05s/it]


epoch,▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆████████
train_loss,▂█▂▂▁▂▂▂▃▁▂▂▂▂▂▂▂▂▂▂▃▂▂▂▂▃▂▂▁▃▂▂▂▂▂▂▁▃▂▂
validation_accuracy,▁▁▁▁▁
validation_loss,█▁▄▃▃
epoch,4
train_loss,2.33305
validation_accuracy,0.11
validation_loss,2.30913


wandb: Agent Starting Run: 1luoxxib with config:
wandb: 	architecture: CNN
wandb: 	batch_size: 192
wandb: 	classes: 10
wandb: 	dataset: MNIST
wandb: 	dropout: 0.3
wandb: 	epochs: 5
wandb: 	kernels: [32, 64]
wandb: 	learning_rate: 0.010762898949392275
wandb: 	log_prediction_iamge: False
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb:   3 of 3 files downloaded.  
100%|██████████| 5/5 [00:34<00:00,  6.99s/it]


epoch,▁▁▁▁▁▁▃▃▃▃▃▃▅▅▅▅▅▅▆▆▆▆▆▆██████
train_loss,▃█▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
validation_accuracy,▁▅▇██
validation_loss,█▆▂▂▁
epoch,4
train_loss,0.26661
validation_accuracy,0.87
validation_loss,0.44034
